# 股價資訊

In [1]:
import requests
from bs4 import BeautifulSoup
from ast import literal_eval
def stock_info(stock_name='2330'):
    # 新聞網址 news_target為相對網址，需要 domain_url
    domain_url = 'https://www.wantgoo.com/'
    stock_url = 'https://www.wantgoo.com/stock/' + stock_name
    # response
    res = requests.get(stock_url)
    # 解析
    soup = BeautifulSoup(res.text, 'html.parser')
    # 目標: 新聞
    news_target = soup.find_all('ul', class_='ell lists')[0].find_all('a')

    news_content = ''
    # 只取 5筆
    if len(news_target)>5:
        num = 5
    else :
        num = len(news_target)
    for i in range(num):
        news_content += f"{news_target[i].text} \n " + \
            f"{domain_url + news_target[i].get('href')} \n\n"
    
    # 即時股價資訊，需要 headers 進行訪問
    head = {
        'referer': stock_url,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
    }

    realtime_url = 'https://www.wantgoo.com/stock/astock/realtimechartupdatedata?StockNo=' + stock_name
    res = requests.get(realtime_url,headers=head)
    
    # eval() :str -> 轉成程式碼 ; res.json()['returnValues'] 回傳文字
    # https://stackoverflow.com/questions/30109030/how-does-strlist-work
    realtime_stock_info = literal_eval(
        res.json()['returnValues'].replace('null','"nan"') # 更換空值 null -> nan
    )['_01_基本股價資訊']

    stock_info_content = ''
    for i in ['更新時間','StockNo','Name','開','高','低','收','成交量']:
        stock_info_content += f'{i}: {realtime_stock_info[i]} \n'
    
    content = '相關新聞:\n' + news_content + '即時股價資訊:\n' + stock_info_content
    return content

In [2]:
content = stock_info(stock_name='1101b')

print(content)

相關新聞:
台泥特別股、光隆特別股掛牌 早盤逆勢上漲 
 https://www.wantgoo.com//News/Content/Index?ID=927167 

即時股價資訊:
更新時間: 2019/03/08 13:30:36 
StockNo: 1101B 
Name: 台泥乙特 
開: 51.4 
高: 51.5 
低: 51.4 
收: 51.5 
成交量: 205 



# 油價資訊

In [3]:
def oil_price():
    url = 'https://gas.goodlife.tw/'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    # 最後更新時間
    main = soup.select('#main')[0].text.replace('\n', '').split('(')[0]
    # 調整
    gas_price = soup.select('#gas-price')[0].text.replace('\n\n\n', '').replace(' ', '')
    # 油價
    cpc = soup.select('#cpc')[0].text.replace(' ', '').split('\n')

    cpc_title = cpc[1] # 今日中油油價
    cpc_92 = cpc[5]
    cpc_95 = cpc[8]
    cpc_98 = cpc[11]
    cpc_柴油 = cpc[14]

    content = f'{main}\n{gas_price}\n\n' + \
        f'{cpc_title}\n 92汽油:{cpc_92}\n 95汽油:{cpc_95}\n 98汽油:{cpc_98}\n 柴油:{cpc_柴油}'

    return content

In [4]:
print(oil_price())

最後更新時間: 2019-03-09 21:40 
柴油預計調整:
-0.1元
受到鄰國油價影響，下週一2019年03月11日起,預計汽油每公升:
不調整

今日中油油價
 92汽油:26.5
 95汽油:28.0
 98汽油:30.0
 柴油:25.5
